In [1]:
import shodan
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from string import Template
import json
import networkx as nx
import re

In [2]:
nodes = set()
edges = set()

for banner in shodan.helpers.iterate_files('isp1k.json.gz'):
    nodes.add(banner['isp'])
    nodes.add(banner['ip_str'])
    edges.add((banner['ip_str'], banner['isp']))

In [3]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.nx_agraph.graphviz_layout(G)

In [4]:
graph = {'nodes': [], 'edges': []}
for node in nodes:
    graph['nodes'].append({
        'id': node,
        'label': node,
        'x': pos[node][0],
        'y': pos[node][1],
        'color': 'black' if re.match(r'^\d+\.', node) else 'red'
    })

for i, edge in enumerate(edges):
    graph['edges'].append({
        'id': i,
        'source': edge[0],
        'target': edge[1],
        'color': 'rgba(200, 200, 200, .5)'
    })

print(len(graph['nodes']))
print(graph)

995
{'nodes': [{'id': '185.151.4.220', 'label': '185.151.4.220', 'x': 1422.0, 'y': 900.08, 'color': 'black'}, {'id': '92.99.57.168', 'label': '92.99.57.168', 'x': 976.16, 'y': 961.68, 'color': 'black'}, {'id': '2.50.54.210', 'label': '2.50.54.210', 'x': 1096.7, 'y': 858.0, 'color': 'black'}, {'id': '94.204.252.235', 'label': '94.204.252.235', 'x': 643.79, 'y': 706.82, 'color': 'black'}, {'id': '86.96.253.52', 'label': '86.96.253.52', 'x': 1036.7, 'y': 1071.2, 'color': 'black'}, {'id': '83.110.85.202', 'label': '83.110.85.202', 'x': 1148.7, 'y': 934.92, 'color': 'black'}, {'id': '92.96.222.77', 'label': '92.96.222.77', 'x': 1052.8, 'y': 988.65, 'color': 'black'}, {'id': '87.247.174.252', 'label': '87.247.174.252', 'x': 1317.4, 'y': 796.64, 'color': 'black'}, {'id': '86.96.109.234', 'label': '86.96.109.234', 'x': 1023.6, 'y': 884.07, 'color': 'black'}, {'id': '185.42.142.51', 'label': '185.42.142.51', 'x': 1453.5, 'y': 124.64, 'color': 'black'}, {'id': '94.59.102.61', 'label': '94.59.102

In [5]:
js_template = Template('''
graph = $graph_data

S = new sigma({
  graph: graph,
  renderer: {
    container: 'container',
    type: 'canvas'
  }
})

sigma.plugins.relativeSize(S, 1)

S.startNoverlap()

S.refresh()
''')

html_template = Template('''
<title>IPs and ISPs in the UAE</title>
<meta charset='utf-8'>
<h1 style='font-family:sans-serif;text-align:center'>$title</h1>
<p style='text-align:center'>Drag, click, and scroll on the map to interact.</p>
<br>
<div id='container' style='height:800px'></div>
<script src='https://cdnjs.cloudflare.com/ajax/libs/sigma.js/1.2.1/sigma.min.js'></script>
<script src='https://cdnjs.cloudflare.com/ajax/libs/sigma.js/1.2.1/plugins/sigma.plugins.relativeSize.min.js'></script>
<script src='https://cdnjs.cloudflare.com/ajax/libs/sigma.js/1.2.1/plugins/sigma.layout.noverlap.min.js'></script>
<script>$js_text</script>
''')

In [6]:
js_text = js_template.substitute({
    'graph_data': json.dumps(graph)
})

h = html_template.substitute({
    'title': 'IPs and ISPs in the UAE',
    'js_text': js_text
})

f = open('index.html', 'w+')
f.write(h)
f.close()

HTML(h)